In [1]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

from scipy.misc import imread, imresize
from sklearn.externals import joblib
import numpy as np 
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

from helpers import convert, show_images, put_boxes, box_boundaries
from featuresourcer import FeatureSourcer
from binaryclassifier import BinaryClassifier
from slider import Slider
from heatmap import HeatMap 

In [2]:
def hot_to_rgb(data):
  data_ = (data - np.min(data)) / (np.max(data) - np.min(data))
  my_cmap = plt.cm.get_cmap('hot')
  img = my_cmap(data_)
  rgb_img = np.dstack((img[:, :, 0], img[:, :, 1], img[:, :, 2]))
  return rgb_img

In [3]:
svc = joblib.load('svc.pkl')
scaler = joblib.load('scaler.pkl')

cls = BinaryClassifier(svc, scaler)

sourcer_params = {
  'color_model': 'hls',                # HLS, HSV
  'bounding_box_size': 64,             #
  'number_of_orientations': 12,        # 6 - 12
  'pixels_per_cell': 8,                # 8, 16
  'cells_per_block': 2,                # 1, 2
  'do_transform_sqrt': True
}

small_img = imread("vehicles/KITTI_extracted/5364.png")
src = FeatureSourcer(sourcer_params, small_img)

slider = Slider(sourcer = src, classifier = cls, increment = 8)

In [4]:
def verbose_pipeline(this_frame):

  for sz, pos in zip(ws, wp):
    bounding_boxes = slider.locate(frame = this_frame, window_size = sz, window_position = pos)
    heatmap.update(bounding_boxes)

  mp, _, _ = heatmap.get()
  labeled_img = heatmap.draw(this_frame)
  
  rgb_img = imresize(hot_to_rgb(mp), 0.25)
                     
  labeled_img[20:200, 20:340] = rgb_img
  
  return labeled_img

In [ ]:
ws = 180, 100, 120, 140, 180, 210
wp = 360, 390, 390, 390, 390, 390

temp_frame = imread("test1.jpg")
heatmap = HeatMap(frame = temp_frame, thresh = 20, memory = 60)

project_output = 'project_video_output_test1.mp4'
clip1 = VideoFileClip("project_video.mp4");
white_clip = clip1.fl_image(verbose_pipeline) 
%time white_clip.write_videofile(project_output, audio = False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))

In [ ]:
ws = 180, 100, 120, 140, 180, 210
wp = 360, 390, 390, 390, 390, 390
temp_frame = imread("test1.jpg")
heatmap = HeatMap(frame = temp_frame, thresh = 25, memory = 60)

project_output = 'project_video_output_test2.mp4'
clip1 = VideoFileClip("project_video.mp4");
white_clip = clip1.fl_image(verbose_pipeline) 
%time white_clip.write_videofile(project_output, audio = False)

[MoviePy] >>>> Building video project_video_output_test2.mp4
[MoviePy] Writing video project_video_output_test2.mp4


  1%|          | 15/1261 [03:09<4:20:09, 12.53s/it]